In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import fastf1 as f1
from matplotlib.collections import LineCollection
import matplotlib as mpl

In [ ]:
year = int(input("Enter year, earliest 2018: "))

if year < 2018:
    raise ValueError("Year must be 2018 or later.")

schedule = f1.get_event_schedule(year)
available_weekends = schedule[['RoundNumber', 'EventName']]

# Print the race weekends in the desired format
print("Available race weekends:")
for _, row in available_weekends.iterrows():
    print(f"{row['RoundNumber']}: {row['EventName']}")

wknd = int(input("Enter race weekend number: "))

if (year == 2025 and wknd > 5) or wknd not in available_weekends['RoundNumber'].values:
    raise ValueError("Weekend number is not valid for the selected year.")


ses = 'R'
colormap = mpl.cm.plasma

session = f1.get_session(year, wknd, ses)
weekend = session.event
session.load()


In [ ]:
# Retrieve and format the list of available drivers
available_drivers = session.laps['Driver'].unique()
available_drivers_str = ', '.join(available_drivers)

print(f"Available drivers: {available_drivers_str}")

In [ ]:
drivers = input("Enter driver codes separated by commas: ").split(",")

drivers = [driver.strip() for driver in drivers]

if not all(driver in available_drivers for driver in drivers):
    raise ValueError("One or more driver codes are invalid. Please check the available drivers list.")

# Assign a unique color to each driver
driver_colors = {driver.strip(): mpl.cm.tab10(i) for i, driver in enumerate(drivers)}

In [ ]:
# Loop through each driver and collect telemetry data
driver_telemetry = {}
for driver in drivers:
    driver = driver.strip()
    lap = session.laps.pick_drivers(driver).pick_fastest()
    driver_telemetry[driver] = {
        "x": lap.telemetry['X'].values,
        "y": lap.telemetry['Y'].values,
        "speed": lap.telemetry['Speed'].values
    }

# Use the first driver's telemetry as the reference
reference_driver = drivers[0].strip()
x_telemetry = driver_telemetry[reference_driver]["x"]
y_telemetry = driver_telemetry[reference_driver]["y"]

# Determine the minimum length of telemetry data across all drivers
min_length = min(len(driver_telemetry[driver]["speed"]) for driver in drivers)

fastest_driver = []

# Determine the fastest driver at each point
for i in range(min_length):
    max_speed = -1
    fastest = None
    for driver in drivers:
        driver = driver.strip()
        speed = driver_telemetry[driver]["speed"][i]
        if speed > max_speed:
            max_speed = speed
            fastest = driver
    fastest_driver.append(fastest)


In [ ]:
points = np.array([x_telemetry, y_telemetry]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

In [ ]:
# Map drivers to colors for the plot
colors = [driver_colors[driver] for driver in fastest_driver]

# Create the plot
fig, ax = plt.subplots(sharex=True, sharey=True, figsize=(12, 6.75))
fig.suptitle(f'{session.event['EventName']} {year} - Fastest Driver - {", ".join(drivers)}', size=24, y=0.97)

# Adjust margins and turn off axis
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.12)
ax.axis('off')

# Plot the background track line
ax.plot(x_telemetry, y_telemetry, color='black', linestyle='-', linewidth=16, zorder=0)

# Create a LineCollection with driver-specific colors
lc = LineCollection(segments, colors=colors, linestyle='-', linewidth=5)

# Add the collection to the plot
ax.add_collection(lc)

# Create a legend for the drivers
legend_handles = [mpl.lines.Line2D([0], [0], color=color, lw=4, label=driver) for driver, color in driver_colors.items()]
ax.legend(handles=legend_handles, loc='upper right', title="Drivers")

# Show the plot 
plt.show()